## End to End Workflow

## Step 1:  Imports and setup

The following is just boilerplate code that sets up the Spark session and sets some other non-essential configuration options

In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType
import pyspark.sql.functions as f
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [2]:
import os
import time
import json
import requests
import xml.etree.ElementTree as ET
import datetime

#Extracting the correct URL from hive-site.xml
tree = ET.parse('/etc/hadoop/conf/hive-site.xml')
root = tree.getroot()

for prop in root.findall('property'):
    if prop.find('name').text == "hive.metastore.warehouse.dir":
        storage = prop.find('value').text.split("/")[0] + "//" + prop.find('value').text.split("/")[2]

print("The correct Cloud Storage URL is: {}".format(storage))

os.environ['STORAGE'] = storage

The correct Cloud Storage URL is: s3a://demo-aws-2


In [3]:
#conf=SparkConf()

# Load in a jar that provides extended string comparison functions such as Jaro Winkler.
# Splink
#     conf.set('spark.driver.extraClassPath', 'jars/scala-udf-similarity-0.0.6.jar,jars/graphframes-0.6.0-spark2.3-s_2.11.jar')
#     conf.set('spark.jars', 'jars/scala-udf-similarity-0.0.6.jar,jars/graphframes-0.6.0-spark2.3-s_2.11.jar')
#conf.set('spark.driver.extraClassPath', 'jars/scala-udf-similarity-0.0.6.jar')
#conf.set('spark.jars', 'jars/scala-udf-similarity-0.0.6.jar')
#conf.set('spark.jars.packages', 'graphframes:graphframes:0.6.0-spark2.3-s_2.11')

#sc = SparkContext.getOrCreate(conf=conf)
#sc.setCheckpointDir("temp_graphframes/")


spark = SparkSession\
    .builder\
    .appName("Entity Resolution with Lineage")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-1")\
    .config("spark.yarn.access.hadoopFileSystems", os.environ['STORAGE'])\
    .config("spark.driver.extraClassPath", "jars/scala-udf-similarity-0.0.6.jar")\
    .config("spark.jars", "jars/scala-udf-similarity-0.0.6.jar")\
    .getOrCreate()

# Register UDFs
from pyspark.sql import types
spark.udf.registerJavaFunction('jaro_winkler_sim', 'uk.gov.moj.dash.linkage.JaroWinklerSimilarity', types.DoubleType())
spark.udf.registerJavaFunction('Dmetaphone', 'uk.gov.moj.dash.linkage.DoubleMetaphone', types.StringType())

In [4]:
spark

In [5]:
import pandas as pd 
pd.options.display.max_columns = 500

In [6]:
import logging 
logging.basicConfig()  # Means logs will print in Jupyter Lab

# Set to DEBUG if you want splink to log the SQL statements it's executing under the hood
logging.getLogger("splink").setLevel(logging.INFO)

## Step 2: Read in the data

The `l` and `r` stand for 'left' and 'right.  It doesn't matter which of the two datasets you choose as the left, performance and results will be the same.

⚠️ Note that `splink` makes the following assumptions about your data:

-  There is a field containing a unique record identifier in each dataset
-  The two datasets being linked have common column names - e.g. date of birth is represented in both datasets in a field of the same name.   In many cases, this means that the user needs to rename columns prior to using `splink`


##### READING FROM PHOENIX INTO A SPARK DF

In [7]:
class Db:
    def __init__(self):
        opts = {}
        opts['authentication'] = 'BASIC'
        opts['avatica_user'] = os.environ["WORKLOAD_USER"]
        opts['avatica_password'] = os.environ["WORKLOAD_PASSWORD"]
        database_url = os.environ["OPDB_ENDPOINT_AWS2"]
        self.TABLENAME = "test_table_paul"
        self.conn = phoenixdb.connect(database_url, autocommit=True,**opts)
        self.curs = self.conn.cursor()
        
    def get_data(self):

        query = f"SELECT * FROM CML_WORKSHOP_TABLE_RIGHT"

        model.curs.execute(query)
        rows = model.curs.fetchall()

        return rows

In [8]:
import logging
logging.basicConfig( level=logging.DEBUG)

import os
import phoenixdb
model = Db()

In [9]:
phoenix_df = model.get_data()

In [10]:
schema = StructType([StructField('unique_id', StringType(), True),
                     StructField('first_name', StringType(), True), 
                     StructField('surname', StringType(), True), 
                     StructField('dob', StringType(), True), 
                     StructField('city', StringType(), True), 
                     StructField('email', StringType(), True), 
                     StructField('group', StringType(), True)])

In [23]:
right_df = spark.createDataFrame(phoenix_df, schema=schema)

In [24]:
right_df = right_df.withColumn("source_dataset", f.lit("df_r"))

##### READING FROM HIVE INTO A SPARK DF

In [25]:
left_df = spark.sql("SELECT * FROM default.mytable")

In [26]:
left_df = right_df.withColumn("source_dataset", f.lit("df_l"))

In [27]:
left_df.show(5)
right_df.show(5)

+---------+----------+-------+----------+--------------+--------------------+-----+--------------+
|unique_id|first_name|surname|       dob|          city|               email|group|source_dataset|
+---------+----------+-------+----------+--------------+--------------------+-----+--------------+
|        0|    Julia |   None|2015-10-29|        London| hannah88@powers.com|    0|          df_l|
|      105|    Harry |  Tomas|2011-07-30|        Belast|sandra26@anderson...|   21|          df_l|
|      115|      None|   Ells|2013-01-20|Stoke-on-Trent|wmcdaniel@nelson.net|   22|          df_l|
|      122| Isabella | Wallca|2000-03-24|        London|hilltheresa@pears...|   23|          df_l|
|      128|   Edward |   wLis|2005-04-09|          None|whitakernichole@b...|   24|          df_l|
+---------+----------+-------+----------+--------------+--------------------+-----+--------------+
only showing top 5 rows

+---------+----------+-------+----------+--------------+--------------------+-----+-

## Step 3:  Configure splink using the `settings` object

Most of `splink` configuration options are stored in a settings dictionary.  This dictionary allows significant customisation, and can therefore get quite complex.  

💥 We provide an tool for helping to author valid settings dictionaries, which includes tooltips and autocomplete, which you can find [here](http://robinlinacre.com/splink_settings_editor/).

Customisation overrides default values built into splink.  For the purposes of this demo, we will specify a simple settings dictionary, which means we will be relying on these sensible defaults.

To help with authoring and validation of the settings dictionary, we have written a [json schema](https://json-schema.org/), which can be found [here](https://github.com/moj-analytical-services/splink/blob/master/splink/files/settings_jsonschema.json).  




In [30]:
# The comparison expression allows for the case where a first name and surname have been inverted 
sql_case_expression = """
CASE 
WHEN first_name_l = first_name_r AND surname_l = surname_r THEN 4 
WHEN first_name_l = surname_r AND surname_l = first_name_r THEN 3
WHEN first_name_l = first_name_r THEN 2
WHEN surname_l = surname_r THEN 1
ELSE 0 
END
"""

settings = {
    "link_type": "link_only", 
    "max_iterations": 20,
    "blocking_rules": [
    ],
    "comparison_columns": [
       {
            "custom_name": "name_inversion",
            "custom_columns_used": ["first_name", "surname"],
            "case_expression": sql_case_expression,
            "num_levels": 5
        },
        {
            "col_name": "city",
            "num_levels": 3
        },
        {
            "col_name": "email",
            "num_levels": 3
        },
        {
            "col_name": "dob"
        }
    ],
    "additional_columns_to_retain": ["group"]
    
}

In words, this setting dictionary says:

- We are performing a data linking task (the other options are `dedupe_only`, or `link_and_dedupe`)
- Rather than generate all possible comparisons (the cartesian product of the input datasets), we are going restrict record comparisons to those generated by at least one of the rules in the specified array
- When comparing records, we will use information from the `first_name`, `surname`, `dob`, `city` and `email` columns to compute a match score.
- For `first_name` and `surname`, string comparisons will have three levels:
    - Level 2: Strings are (almost) exactly the same
    - Level 1: Strings are similar 
    - Level 0: No match
- We will make adjustments for term frequencies on the `first_name` and `surname` columns
- We will retain the `group` column in the results even though this is not used as part of comparisons.  This is a labelled dataset and `group` contains the true match - i.e. where group matches, the records pertain to the same person

## Step 4:  Estimate match scores using the Expectation Maximisation algorithm

In [31]:
from splink import Splink

linker = Splink(settings, [left_df, right_df], spark)
df_linked = linker.get_scored_comparisons()

/home/cdsw/.local/lib/python3.6/site-packages/splink/default_settings.py:175: UserWarning: You have not specified any blocking rules, meaning all comparisons between the input dataset(s) will be generated and blocking will not be used.For large input datasets, this will generally be computationally intractable because it will generate comparisons equal to the number of rows squared.
  "You have not specified any blocking rules, meaning all comparisons between the "
INFO:splink.iterate:Iteration 0 complete
INFO:splink.model:The maximum change in parameters was 0.3704400897026062 for key name_inversion, level 0
INFO:splink.iterate:Iteration 1 complete
INFO:splink.model:The maximum change in parameters was 0.0720943808555603 for key name_inversion, level 4
INFO:splink.iterate:Iteration 2 complete
INFO:splink.model:The maximum change in parameters was 0.013827092945575714 for key dob, level 0
INFO:splink.iterate:Iteration 3 complete
INFO:splink.model:The maximum change in parameters was 0.

In [32]:
# Inspect main dataframe that contains the match scores
df_linked.toPandas().sample(5)

,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,first_name_l,first_name_r,surname_l,surname_r,gamma_name_inversion,city_l,city_r,gamma_city,email_l,email_r,gamma_email,dob_l,dob_r,gamma_dob,group_l,group_r
7816,7.898760e-61,df_l,558,df_r,49,Zachary,Theodore,Namen,None,0,London,Leeds,0,jessicapowell@hardin-mcgee.net,markstewart@gross.com,0,1987-04-03,1982-09-10,0,94,12
4979,7.898760e-61,df_l,394,df_r,261,None,Florence,Davis,Morris,0,Ipswich,Southampton,0,jose33@trujillo.net,paul49@allen.biz,0,2001-07-13,1993-01-29,0,67,45
20583,7.898760e-61,df_l,799,df_r,431,Benjamin,Lucy,Kelly,Stone,0,Loodn,Middbeslough,0,joshua10@werner.com,ppruitt@kirby.org,0,2018-12-20,2011-11-26,0,141,73
26922,7.898760e-61,df_l,714,df_r,924,Hughes,Mills,Arthur,Thomas,0,Sheffield,London,0,eileenlopez@mcmahon-long.info,hensondebbie@garcia.com,0,1998-08-24,1970-03-09,0,121,167
15939,7.898760e-61,df_l,558,df_r,644,Zachary,Daniel,Namen,Palmer,0,London,oodLn,0,jessicapowell@hardin-mcgee.net,None,0,1987-04-03,2016-06-14,0,94,109


In [94]:
df_linked.write.format("parquet").mode("overwrite").saveAsTable('default.matches_spark_table')

##### UPLOADING DATA TO IMPALA

Preparing data before insertion

In [33]:
df_impala = df_linked.toPandas()

In [34]:
df_impala.head()

,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,first_name_l,first_name_r,surname_l,surname_r,gamma_name_inversion,city_l,city_r,gamma_city,email_l,email_r,gamma_email,dob_l,dob_r,gamma_dob,group_l,group_r
0,1.000000e+00,df_l,0,df_r,0,Julia,Julia,None,None,4,London,London,2,hannah88@powers.com,hannah88@powers.com,2,2015-10-29,2015-10-29,1,0,0
1,7.898760e-61,df_l,0,df_r,105,Julia,Harry,None,Tomas,0,London,Belast,0,hannah88@powers.com,sandra26@anderson-davis.com,0,2015-10-29,2011-07-30,0,0,21
2,7.898760e-61,df_l,0,df_r,115,Julia,None,None,Ells,0,London,Stoke-on-Trent,0,hannah88@powers.com,wmcdaniel@nelson.net,0,2015-10-29,2013-01-20,0,0,22
3,8.285076e-35,df_l,0,df_r,122,Julia,Isabella,None,Wallca,0,London,London,2,hannah88@powers.com,hilltheresa@pearson.org,0,2015-10-29,2000-03-24,0,0,23
4,7.898760e-61,df_l,0,df_r,128,Julia,Edward,None,wLis,0,London,None,0,hannah88@powers.com,whitakernichole@booth.com,0,2015-10-29,2005-04-09,0,0,24


##### We round match probabilities and pick entities that are a match

In [47]:
df_impala = df_impala.round(1)

In [51]:
df_impala = df_impala[df_impala["match_probability"] == 1]

In [54]:
#Filtering duplicate columns
df_impala = df_impala.filter(regex="_l")

In [56]:
df_impala.columns = df_impala.columns.str.replace("_l", "")

###### We are ready to upsert our matches into Impala

In [73]:
import os
import jaydebeapi
conn = jaydebeapi.connect("com.cloudera.impala.jdbc.DataSource",
                          "jdbc:impala://"+os.environ["IMPALA_HOST"]+":443/;ssl=1;transportMode=http;httpPath=cliservice;AuthMech=3;",
                          {'UID': os.environ["WORKLOAD_USER"], 'PWD': os.environ["WORKLOAD_PASSWORD"]},
                          '/home/cdsw/impala_drivers/ImpalaJDBC41.jar')

In [74]:
cursor = conn.cursor()

In [75]:
query = """
        CREATE TABLE IF NOT EXISTS ER_MATCHES (
        unique_id VARCHAR,
        first_name VARCHAR,
        surname VARCHAR,
        dob VARCHAR,
        city  VARCHAR,
        email VARCHAR,
        groupP VARCHAR)
        """

In [76]:
cursor.execute(query)

In [77]:
def upsert_linkage(data):

    sql = """insert into ER_MATCHES \
         (unique_id ,first_name,surname,dob,city,email,groupP) \
         values (?,?,?,?,?,?,?)"""
    #print(data)
    cursor.executemany(sql,data)

In [78]:
def upsert_data_jaydebeapi(data, records=100):
    total_records=0
    header = True
    rows = []
    i=1

    for index, row in data.iterrows():

        rows.append ([f"{row['unique_id']}",\
                  f"{row['first_name']}",f"{row['surname']}",\
                  f"{row['dob']}",f"{row['city']}", \
                  f"{row['email']}", f"{row['group']}"])
        total_records=total_records+1

        if i < records + 1 :   
            i=i+1
        else :
            upsert_linkage(rows)
            rows = []
            i=1
            print (f"Ingested {total_records} records")

    if len(rows) > 0 :
        upsert_linkage(rows)

    print (f"Ingested {total_records} records")

In [79]:
upsert_data_jaydebeapi(df_impala)

#curs.fetchall()

#cursor.close()
#conn.close()

Ingested 101 records
Ingested 181 records


## Step 5: Create a Custom Atlas Type (Process) reflecting the EM algorithm

First we need to instantiate the connection to Atlas in CDP

In [80]:
import atlasclient

Endpoint, Username and Passoword are stored as CML project variables and passed dynamically

In [82]:
from atlasclient.client import Atlas
client = Atlas(os.environ["ATLAS_ENDPOINT"], port='', username=os.environ["WORKLOAD_USER"], password=os.environ["WORKLOAD_PASSWORD"])

Verify the Client connection is working by querying a random Atlas entity

In [83]:
guid = "c845eb62-d85d-4591-8abe-0c31449cdd95"

In [84]:
entity = client.entity_guid(guid)

In [85]:
entity.entity['attributes']

ERROR:atlasclient.base:Missing attr EntityGuid: searchParameters


{'createTime': 1607119259591,
 'description': None,
 'displayName': None,
 'metadata': {'default_project_engine_type': 'legacy_engine',
  'project_description': 'Build an XGBoost model to predict churn using customer telco data.',
  'project_visibility': 'private',
  'service_name': 'mlgov/mlgovapiserver.mlx.cloudera.site@DEMO-AWS.YLCU-ATMI.CLOUDERA.SITE',
  'session_username': 'alexbleakley'},
 'modifiedTime': 1607119259591,
 'name': 'Churn Modeling with XGBoost - alexbleakley',
 'owner': 'alexbleakley',
 'qualifiedName': 'crn:cdp:ml:us-west-1:8a1e15cd-04c2-48aa-8f35-b4a8c11997d3:workspace:ab34ab61-368e-46a2-923e-6c5830585734/affaa215-c794-436c-af69-dcccfffb3d02',
 'replicatedFrom': None,
 'replicatedTo': None,
 'userDescription': None}

Looks like we have successfully established the connection. Next we can create a custom Atlas type (process) reflecting the EM algorithm

In [86]:
typedef_dict = {
    "enumTypes": [],
    "structTypes": [],
    "classificationDefs":[],
    "entityDefs": [{
        "superTypes": ["Process"],
        "name": "ER_algorithm",
        "description":"custom_type_for_Entity_Resolution",
        "attributeDefs": [{
            "name": "startTime",
            "isOptional": True,
            "isUnique": False,
            "isIndexable": False,
            "typeName":"string",
            "valuesMaxCount":1,
            "cardinality":"SINGLE",
            "valuesMinCount":0
        }]
    }]
}

And we can now register the new type with Atlas. For more on the Atlas type model, please visit this page: https://docs.cloudera.com/runtime/7.2.7/cdp-governance-overview/topics/atlas-metadata-model-overview.html

In [87]:
#Has already run once so will not run again
client.typedefs.create(data=typedef_dict)

## Step 6: Instantiate the EM algorithm in Atlas along with lineage reflecting our Linkage Job above

Notice: we need to pass the Atlas guid for the two datasets we compared above as they were registered in Atlas when they were stored as a Spark table

In [37]:
#Retrieving GUID's for the three tables via Atlas Client - search by name

In [93]:
params = {'typeName': 'hbase_table', 'attrName': 'data', 'attrValue': 'provider','offset': '1', 'limit':'100'}
search_results = client.search_basic(**params)
for s in search_results:
    for e in s.entities:
        print(e.guid)
        print(e.attributes)
        #print(e.attributes.values)
        print(e.typeName)
        print(e.attributes)

a1fb2bf9-c720-4041-8281-f819ca22dc93
{'owner': 'jmarchand', 'createTime': 1612185801515, 'qualifiedName': 'SIMPLE:US_POPULATION@cm', 'name': 'SIMPLE:US_POPULATION', 'description': 'SIMPLE:US_POPULATION'}
hbase_table
{'owner': 'jmarchand', 'createTime': 1612185801515, 'qualifiedName': 'SIMPLE:US_POPULATION@cm', 'name': 'SIMPLE:US_POPULATION', 'description': 'SIMPLE:US_POPULATION'}
47fb76d7-c725-467e-b7e5-e9a44069148a
{'owner': 'jmarchand', 'createTime': 1612187145196, 'qualifiedName': 'default:FR_POPULATION@cm', 'name': 'FR_POPULATION', 'description': 'FR_POPULATION'}
hbase_table
{'owner': 'jmarchand', 'createTime': 1612187145196, 'qualifiedName': 'default:FR_POPULATION@cm', 'name': 'FR_POPULATION', 'description': 'FR_POPULATION'}
4fd2817b-fc13-47e2-aa54-70255a22ea25
{'owner': 'jmarchand', 'createTime': 1612186479256, 'qualifiedName': 'default:US_POPULATION2@cm', 'name': 'US_POPULATION2', 'description': 'US_POPULATION2'}
hbase_table
{'owner': 'jmarchand', 'createTime': 1612186479256, 'q

In [39]:
params = {'typeName': 'hive_table', 'attrName': 'name', 'attrValue': 'cc_data', 'offset': '1', 'limit':'10'}
search_results = client.search_attribute(**params)
for s in search_results:
    for e in s.entities:
        print(e.guid)
        print(e.attributes)

In [40]:
for s in search_results:
    print(s.entities.to_dict())

In [41]:
data = {'typeName': 'hive_table', 'attrName': 'name', 'attrValue': 'cc_data', 'offset': '1', 'limit': '100'}
search_results = client.search_basic.create(data=data)
for e in search_results.entities:
    print(e.guid)
    print(e.attributes)

##### We can now create a new process type in Atlas reflecting the ER Algorithm 

In [95]:
process_entity_dict = {
  "entity" : {
    "guid" : "-2089428075574333",
    "status" : "ACTIVE",
    "createdBy" : "pdefusco",
    "updatedBy" : "pdefusco",
    "createTime" : "12342",
    "updateTime" : "12342",
    "version" : "12342",
    "relationshipAttributes" : {},
    "classifications" : [],
    "typeName" : "ER_algorithm",
    "attributes" : {
      "startTime" : "123",
      "qualifiedName": "EM Record Linkage",
      "name":"EM Record Linkage",
      "description":"Record Linkage Algorithm",
      "owner": "pdefusco",
        #, 
      "inputs":[{"guid": "2bcde123-67f2-4ed7-a781-4824085a2d90", "typeName":"hive_table"},
               {"guid": "e26109c4-b70a-4583-a361-3de4539f532f", "typeName":"hbase_table"}], 
      "outputs":[{"guid":"ae778e6b-57f4-486e-9643-7ae574a9e16b", "type_name":"hive_table"}]
    }, 
  },
  
}

In [96]:
client.entity_post.create(data=process_entity_dict)

{'guidAssignments': {'-2089428075574333': '19d036e1-a2d5-47f4-89a6-701cc591212a'},
 'mutatedEntities': {'CREATE': [{'attributes': {'description': 'Record Linkage Algorithm',
     'name': 'EM Record Linkage',
     'owner': 'pdefusco',
     'qualifiedName': 'EM Record Linkage'},
    'classificationNames': [],
    'classifications': [],
    'displayText': 'EM Record Linkage',
    'guid': '19d036e1-a2d5-47f4-89a6-701cc591212a',
    'isIncomplete': False,
    'labels': [],
    'meaningNames': [],
    'meanings': [],
    'status': 'ACTIVE',
    'typeName': 'ER_algorithm'}],
  'UPDATE': [{'attributes': {'createTime': 1617238814000,
     'name': 'matches_spark_table',
     'owner': 'pauldefusco',
     'qualifiedName': 'default.matches_spark_table@cm'},
    'classificationNames': [],
    'displayText': 'matches_spark_table',
    'guid': 'ae778e6b-57f4-486e-9643-7ae574a9e16b',
    'isIncomplete': False,
    'labels': [],
    'meaningNames': [],
    'meanings': [],
    'status': 'ACTIVE',
    'ty

##### Go back to the CDP Homepage, then Data Catalog -> Atlas and browse for "ER_algorithm". Open the instance and navigate to the "Lineage" tab 

![title](images/EM_Record_Linkage.png)

## Step 8: Create a new Atlas Process Type related to writing a Spark Dataset to Impala and Instantiate it with source and target

In [ ]:
typedef_dict = {
    "enumTypes": [],
    "structTypes": [],
    "classificationDefs":[],
    "entityDefs": [{
        "superTypes": ["Process"],
        "name": "Write_to_Impala",
        "description":"write_to_impala",
        "attributeDefs": [{
            "name": "startTime",
            "isOptional": True,
            "isUnique": False,
            "isIndexable": False,
            "typeName":"string",
            "valuesMaxCount":1,
            "cardinality":"SINGLE",
            "valuesMinCount":0
        }]
    }]
}

In [ ]:
process_entity_dict = {
  "entity" : {
    "guid" : "-2089428075574888",
    "status" : "ACTIVE",
    "createdBy" : "pdefusco",
    "updatedBy" : "pdefusco",
    "createTime" : "12342",
    "updateTime" : "12342",
    "version" : "12342",
    "relationshipAttributes" : {},
    "classifications" : [],
    "typeName" : "Write_to_Impala",
    "attributes" : {
      "startTime" : "123",
      "qualifiedName": "Write_to_Impala",
      "name":"Write_to_Impala",
      "description":"Record Linkage Algorithm",
      "owner": "pdefusco",
        #, 
      "inputs":[{"guid": "ae778e6b-57f4-486e-9643-7ae574a9e16b", "typeName":"hive_table"}], 
      "outputs":[{"guid":"INSERT IMPALA TABLE GUID HERE", "type_name":"IMPALA_TABLE???"}]
    }, 
  },
  
}

![title](images/ER_atlas_lineage.png)

Next we can optionally remove the EM Algorithm instance from Atlas via the client

In [ ]:
entity = client.entity_guid("44848fe5-6950-4a73-a89c-9775b736b4c9")

In [ ]:
entity.entity['attributes']["owner"]

In [ ]:
entity.delete()

## We have completed our introduction to Splink and the Atlas Client. 
## Next we will simulate a real world Application with CML Jobs and COD (Cloudera Operational Database)